In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

import seaborn as sns
import random as rn
import copy as copy

import time

In [2]:
df = pd.read_csv('imbd_cleaned.csv')

X = df.select_dtypes(include=['number'])
X = X.drop(['averageRating'], axis = 1)
X = X.values
y = np.array(df['rating'])

FileNotFoundError: [Errno 2] No such file or directory: 'imbd_cleaned.csv'

## Data Partitioning

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=100, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
clf = HistGradientBoostingClassifier(learning_rate=1.0, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.36725127837086213
F1-score [0.         0.         0.00516796 0.00383509 0.02125399 0.27648202
 0.32591205 0.52907553 0.09114359 0.03255485]


c:\Users\Lavigi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lavigi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      (0, 1]       0.00      0.00      0.00        26
      (1, 2]       0.00      0.00      0.00       141
      (2, 3]       0.03      0.00      0.01       353
      (3, 4]       0.12      0.00      0.00      1027
      (4, 5]       0.29      0.01      0.02      2719
      (5, 6]       0.29      0.27      0.28      6372
      (6, 7]       0.32      0.33      0.33     11643
      (7, 8]       0.42      0.72      0.53     14463
      (8, 9]       0.24      0.06      0.09      6563
     (9, 10]       0.17      0.02      0.03      1281

    accuracy                           0.37     44588
   macro avg       0.19      0.14      0.13     44588
weighted avg       0.32      0.37      0.31     44588



c:\Users\Lavigi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# XGBoost
https://xgboost.readthedocs.io/en/stable/python/python_intro.html

In [ ]:
# !pip install xgboost

  Using cached xgboost-3.0.0-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.0-py3-none-win_amd64.whl (150.0 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np

def convert_interval_to_numeric(interval):
    # Rimuovi le parentesi e dividi per la virgola
    lower, upper = interval[1:-1].split(', ')
    # Converti in float e calcola il punto medio
    return int(lower)

# Applica la conversione a y_train e y_test
y_train_numeric = np.array([convert_interval_to_numeric(interval) for interval in y_train])
y_test_numeric = np.array([convert_interval_to_numeric(interval) for interval in y_test])

print("y_train convertito:", y_train_numeric)
print("y_test convertito:", y_test_numeric)


y_train convertito: [6 7 7 ... 6 8 6]
y_test convertito: [8 7 6 ... 5 4 5]


In [ ]:
from xgboost import XGBClassifier

In [ ]:

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_curve, precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt

# Definisci il modello
clf = XGBClassifier(objective='binary:logistic', tree_method='exact', use_label_encoder=True, random_state=42)

# Definisci la griglia di parametri
param_dist = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [50, 100, 200, 300],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_lambda': [0, 1, 10],
    'reg_alpha': [0, 0.1, 1]
}

# Imposta la grid search con early stopping
fit_params = {
    'early_stopping_rounds': 10,
    'eval_metric': 'logloss',
    'eval_set': [(X_test, y_test_numeric)],
    'verbose': False
}

n_iter = 30

random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, scoring='accuracy',
                                   cv=5, verbose=2, n_jobs=-1, n_iter=n_iter)

# Esegui la grid search con early stopping
random_search.fit(X_train, y_train_numeric, **fit_params)

# Stampa i migliori parametri
print("Migliori parametri trovati:", random_search.best_params_)
print("Miglior punteggio:", random_search.best_score_)


c:\Users\Lavigi\programmi\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:27:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy 0.5133443975957657
F1-score [0.37837838 0.27       0.22692308 0.2609216  0.36440849 0.41377634
 0.4602241  0.6467887  0.47157191 0.54801325]
              precision    recall  f1-score   support

           0       0.64      0.27      0.38        26
           1       0.46      0.19      0.27       141
           2       0.35      0.17      0.23       353
           3       0.34      0.21      0.26      1027
           4       0.40      0.33      0.36      2719
           5       0.43      0.40      0.41      6372
           6       0.46      0.46      0.46     11643
           7       0.59      0.71      0.65     14463
           8       0.52      0.43      0.47      6563
           9       0.58      0.52      0.55      1281

    accuracy                           0.51     44588
   macro avg       0.48      0.37      0.40     44588
weighted avg       0.50      0.51      0.51     44588



In [ ]:
# K-fold cross validation
kfold = StratifiedKFold(n_splits=5)
results = cross_val_score(random_search.best_estimator_, X_train, y_train_numeric, cv=kfold)
print("Risultati della K-fold cross validation:", results)
print("Media dei risultati della K-fold cross validation:", results.mean())

In [ ]:
# Calcola il numero medio di epoch del miglior modello
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train_numeric)
evals_result = best_model.evals_result()
epochs = len(evals_result['validation_0']['logloss'])
print("Numero medio di epoch del miglior modello:", epochs)

In [ ]:
# Calcola la ROC curve e la Precision-Recall curve
y_pred_proba = best_model.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test_numeric, y_pred_proba)
precision, recall, _ = precision_recall_curve(y_test_numeric, y_pred_proba)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()